In [1]:
import pandas as pd
import os
import requests
import time
from bs4 import BeautifulSoup as bs
from selenium import webdriver
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import json

company = pd.read_csv('./data/glassdoor_url.csv')
company.head(3)

,rank,company,website,URL
0,1,Walmart,https://www.walmart.com/,https://www.glassdoor.com/Overview/Working-at-...
1,2,Amazon,https://www.amazon.com/,https://www.glassdoor.com/Overview/Working-at-...
2,3,Apple,https://www.apple.com/,https://www.glassdoor.com/Overview/Working-at-...


In [2]:
def saveFile(start, end, glassdoor):
    filePath = './data/data/'
    fileNameCSV = 'glassdoor_%03d_%03d.csv'%(start,end)
    pd.DataFrame.from_records(glassdoor).to_csv(filePath+fileNameCSV, index=True)

    fileNameJSON = 'glassdoor_%03d_%03d.json'%(start,end)
    with open(filePath+fileNameJSON, 'w') as fp:
        json.dump(glassdoor, fp, indent=2)

In [3]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("headless")

start = 0
new = np.arange(start,550,40)

glassdoor = {}
for i, url in enumerate(company['URL'][start:]):
    i += start
    print(i, company.loc[i,'company'])
    if i in new:
        driver = webdriver.Chrome('/Users/heejipark/Documents/Knowledge-Graph/DSCI558_Project/chromedriver', options=chrome_options)    
    
    driver.get(url)
    try:
        button = driver.find_element(By.XPATH, '//*[@id="EIOverviewContainer"]/div/div/div/span/button').click()
    except:
        print("no longer description")
    time.sleep(0.5)
    html = driver.page_source
    soup = bs(html)

    value = {}
    div_info = ["headquarters", "size", "founded", "type", "revenue"]
    a_info = ["website", "industry"]
    
    for info in div_info:
        try:
            data = soup.find_all("div", {"data-test": 'employer-'+info})[0].get_text()
            value[info] = data
        except:
            print("no %s"%info)
        
    for info in a_info:
        try:
            data = soup.find_all("a", {"data-test": 'employer-'+info})[0].get_text()
            value[info] = data
        except:
            print("no %s"%info)

    try:
        data = soup.find_all("span", {"data-test": "employerDescription"})[0].get_text()
        value['Description'] = data
    except:
            print("no employerDescription")
            
    glassdoor[company.loc[i,'company']] = value
    if i+1 in new:
        driver.quit()
driver.quit()

487 Dillard's


/var/folders/gz/0lwcd8ys1j75crw0gmb53xqh0000gn/T/ipykernel_5796/913906687.py:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/Users/heejipark/Documents/Knowledge-Graph/DSCI558_Project/chromedriver', options=chrome_options)


488 Regions Financial
no longer description
489 Yum Brands
490 Landstar System
491 CF Industries Holdings
492 Northern Trust
493 Robert Half International
494 Compass
495 Hasbro
496 Roper Technologies
497 ARKO
no longer description
no industry
no employerDescription
498 Frontier Communications
499 Ameren


In [6]:
glassdoor

{"Dillard's": {'headquarters': 'Little Rock, AR',
  'size': '10000+ Employees',
  'founded': '1938',
  'type': 'Company - Public (DDS)',
  'revenue': '$5 to $10 billion (USD)',
  'website': 'www.dillards.com',
  'industry': 'Department, Clothing & Shoe Stores',
  'Description': "Dillard's, Inc. ranks among the nation's largest fashion apparel, cosmetics and home furnishings retailers with annual sales exceeding $6.5 billion. The Company focuses on delivering maximum fashion and value to its shoppers by offering compelling selections complemented by exceptional customer care. Dillard's stores offer a broad selection of merchandise and feature products from both national and exclusive brand sources. The Company operates 278 Dillard's locations and 20 clearance centers spanning 29 states plus an Internet store at www.dillards.com."},
 'Regions Financial': {'headquarters': 'Birmingham, AL',
  'size': '10000+ Employees',
  'founded': '1971',
  'type': 'Company - Public (RF)',
  'revenue': '

In [7]:
saveFile(start,500,glassdoor)